In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [3]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [4]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [5]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [6]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [312]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 1
decimate = 10
drive_freq = 139
fsamp = 5000
def func1_scale(i):
    bb = step_file_objs[i]
        
    freq2 = np.fft.rfftfreq(len(bb.other_data[5]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[5]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.other_data[3])*np.sqrt(2)*100)

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-10000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     print(m1_tmp.values[2]-phi_tmp2)
    return m1_tmp.values[0]

In [313]:
func1_scale(0)

[ 5.6511399] 0.108518384438


-10000.0

In [326]:
step_cal_dir = r'/data/old_trap/20211001/bead1/neutrality6/check_charge_both_ratio_y_redo/'
max_file = 500

In [327]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration

step_file_objs = []
for filname in step_cal_files[:]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

5


In [328]:
amps2 = []
for i in range(num_calib_files):
    print(i,'/',len(step_cal_files))
    amps2.append(func1_scale(i))

0 / 5
[ 4.36732124] 96.3314281675
1 / 5
[ 5.50236155] 96.3319090296
2 / 5
[ 3.084879] 96.3312877896
3 / 5
[ 3.21440624] 96.3316183651
4 / 5
[ 2.67455558] 96.3316819135


In [329]:
amps = np.array(amps)
amps2 = np.array(amps2)
# amps1 = np.array(amps1)
_,ax = plt.subplots(figsize=(9.5,4))
# scale = 375

# Perrinf
# ax.scatter(range(len(amps3)), amps3/scaleY, label='fit phase', alpha=0.3)
ax.scatter(range(len(amps2)), amps2/scaleY, label='fit phase')
ax.scatter(range(len(amps)), amps/scaleY, label='fit phase', alpha=0.4)
# ax.scatter(range(len(amps1)), amps1/scale, label='fit phase', alpha=0.8)

# ax.set(xlabel='dataset', ylabel='electron charges', ylim=(-0.5,6), xlim=(None,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator

ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# scale = -1*np.abs(np.mean(amps3[200:250]))/2
# scale = -1*np.abs(np.mean(amps[230:240]))/3
# scale = -1*np.abs(np.mean(amps[97:118]))/4
scaleY = np.abs(np.mean(amps[150:]))/4
print("Scales are ", scaleY)
bb = step_file_objs[-1]
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[5])
voltage = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage)

Scales are  93.2995440303
voltage:  24.0265126779


In [24]:
scaleY = 93.2995440303
scaleZ = scaleY*(418+627)/(392+406)
scaleX = scaleY*(421+368)/(392+406)
scaleX, scaleY, scaleZ

(92.24729353371767, 93.2995440303, 122.17797432539285)

### Look at neutrality data

In [25]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 1
decimate = 2
def get_amp(i):
    
    fname = r'/data/old_trap/20211001/bead1/neutrality5/Y_175Vpp_alternate_139Hz_spin_XZ_77kHz_no_delay_no_height_fb/Y_175Vpp_'+str(i)+'.h5'
#     fname = r'/data/old_trap/20211001/bead1/discharge/recharge_20211013/Y_50Vpp_'+str(i)+'.h5'

    drive_freq = 139
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[5+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[5+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

#     fit_kwargs = {'A': 0.23, 'f': drive_freq, 'phi': 0, 
#                   'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
#                   'limit_A': [0, 1], 'fix_f': True, 'fix_phi': False,
#                   'print_level': 0}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
#     xx2 = signal.filtfilt(b, a, bb.other_data[5+jj])[::decimate]    
#     m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     phi_tmp = m1_tmp.values[2]

#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*100, np.std(xx2)*np.sqrt(2)*100)
    
    if jj==0:
        phase_diff = 0.12
    else:
        phase_diff = 0.05
            
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma':[100,30000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}

    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = drive_freq*2
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+np.pi/2+phase_diff, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}

        
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[1])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    #     print(m1_tmp.values)
    return m1_tmp.values[0], m2_tmp.values[0], m1_tmp.values[3], m1_tmp.fval

In [247]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 1
decimate = 2
def get_ampZ(i):
    
#     fname = r'/data/old_trap/20211001/bead1/neutrality5/Z_125Vpp_alternate_139Hz_spin_XZ_93kHz_no_delay_no_height_fb/Z_125Vpp_'+str(i)+'.h5'
    fname = r'/data/old_trap/20211001/bead1/neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb_redo2/Z_200Vpp_'+str(i)+'.h5'
#     fname = r'/data/old_trap/20211001/bead1/both_electrodes/Z_139Hz_20Vpp/Z_139Hz_20Vpp_'+str(i)+'.h5'

    drive_freq = 139
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[1+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

    #     fit_kwargs = {'A': 0.23, 'f': drive_freq, 'phi': 0, 
#                   'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
#                   'limit_A': [0, 1], 'fix_f': True, 'fix_phi': False,
#                   'print_level': 0}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
#     xx2 = signal.filtfilt(b, a, bb.other_data[5+jj])[::decimate]    
#     m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     phi_tmp = m1_tmp.values[2]

#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*100, np.std(xx2)*np.sqrt(2)*100)
    
    if jj==0:
        phase_diff = 4.8
    else:
        phase_diff = 4.8
            
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma':[100,3000000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}

    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = drive_freq*2
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+phase_diff+np.pi/2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-1000000, 1000000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}

        
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[2])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    return m1_tmp.values[0], m2_tmp.values[0], m1_tmp.values[3], m1_tmp.fval

In [248]:
i=0
get_ampZ(i)[0]

-11.234170022638864

In [215]:
i = 0
fname = r'/data/old_trap/20211001/bead1/neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb_redo/Z_200Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
print("voltage: ", voltage, np.mean(bb.other_data[1])*100, np.mean(bb.other_data[2])*100)
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
voltage_200_1 = np.std(xx2)*np.sqrt(2)*100
xx3 = signal.filtfilt(b, a, bb.other_data[2])
voltage_200_2 = np.std(xx3)*np.sqrt(2)*100
print("voltage: ", voltage_200_1, voltage_200_2)

voltage:  24.0265126779 -0.587869077592 -0.576897230488
voltage:  190.891726265 0.673523507495


In [30]:
voltage, voltage_150_1

(24.026512677896338, 143.1879022780013)

In [184]:
# amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay = []
# for i in range(1000):
#     print(i,'/500')
#     try:
#         amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay.append(get_amp(i))
#     except:
#         print("******")
# amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay = np.array(amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay)

In [155]:
amps_with_fb_spin_xy_150Vpp_103kHz_z_150Vpp_alternate_no_delay = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i) for i in tqdm(range((20)))))

100%|██████████| 20/20 [00:00<00:00, 17246.32it/s]


In [31]:
def get_amp_wraper(i):
    try:
        return get_amp(i)
    except:
        return 0,0,0,0
    
def get_amp_wraperZ(i):
    try:
        return get_ampZ(i)
    except:
        return 0,0,0,0

In [231]:
amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraper)(i) for i in tqdm(range((500)))))


100%|██████████| 500/500 [00:08<00:00, 57.57it/s]

In [698]:
i=0; j=0
_,ax = plt.subplots()
dat = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[::2]/scaleZ*voltage/voltage_150_1
dat1 = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[1::2]/scaleZ*voltage/voltage_150_1
ax.scatter((dat[:,i]), (dat1[:,j]))
ax.scatter(dat[:,i], np.abs(dat[:,1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
i=0; j=3
_,ax = plt.subplots()
dat = amps_all[::2]/scaleZ*voltage/voltage_150_1
ax.scatter(dat[:,i], np.abs(dat[:,j]))
dat1 = amps_all[1::2]/scaleZ*voltage/voltage_150_1
ax.scatter(dat1[:,i], np.abs(dat1[:,j]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [530]:
voltage= 25

In [531]:
j = 0
data = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[::2,j]/scaleY*voltage/voltage_175_1
data2 = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[1::2,j]/scaleY*voltage/voltage_175_1
data3 = data-0.96*data2

data_1 = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[::2,1]/scaleY*voltage/voltage_175_1
data2_1 = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_175Vpp_alternate_no_delay[1::2,1]/scaleY*voltage/voltage_175_1
data3_1 = data_1+data2_1

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data, label='175Vpp-y-plus, xz-spin-100Vpp-77kHz')
ax.scatter(range(len(data2)), data2, label='175Vpp-y-minus, xz-spin-100Vpp-77kHz', alpha=0.6)
# ax.scatter(range(len(data3)), data3, label='100Vpp-y-diff, xz-spin-100Vpp-77kHz', alpha=0.6)

# ax.scatter(range(len(data4)), data4, label='20V-x, xy-spin-30V, w/o height fb')
ax.plot(np.arange(0,250), np.arange(0,250)*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean, std: ', np.mean(data), np.std(data)/np.sqrt(len(data)))
print('mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))
print('mean, std: ', np.mean(data3), np.std(data3)/np.sqrt(len(data3)))
print('mean, std: ', np.mean(data_1), np.std(data_1)/np.sqrt(len(data_1)))
print('mean, std: ', np.mean(data2_1), np.std(data2_1)/np.sqrt(len(data2_1)))
print('mean, std: ', np.mean(data3_1), np.std(data3_1)/np.sqrt(len(data3_1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean, std:  -0.00255278315035 0.000796258659538
mean, std:  -0.00607773982869 0.000788773392353
mean, std:  0.00328184708519 0.000782182135763
mean, std:  -0.0106432175811 0.00777833856286
mean, std:  0.00302278763124 0.0076577321529
mean, std:  -0.00762042994982 0.0105163138211


In [435]:
scaleZ, voltage, voltage_125_1

(128.33225329571428, 24.026512677896338, 119.33286771283258)

In [743]:
amps_all = list(amps_with_fb_spin_xz_100Vpp_93kHz_z_125Vpp_alternate_no_delay)+list(amps_with_fb_spin_xz_100Vpp_93kHz_z_125Vpp_alternate_no_delay_2)
# amps_all = list(amps_with_fb_spin_xz_100Vpp_93kHz_z_125Vpp_alternate_no_delay_2)
amps_all = np.array(amps_all)
len(amps_all)

5500

In [249]:
amps_with_fb_spin_xy_150Vpp_103kHz_z_200Vpp_alternate_no_delay_redo2 = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i) for i in tqdm(range((500)))))

100%|██████████| 500/500 [00:15<00:00, 32.81it/s]


In [239]:
amps_all = list(amps_with_fb_spin_xy_150Vpp_103kHz_z_200Vpp_alternate_no_delay)
amps_all += list(amps_with_fb_spin_xy_150Vpp_103kHz_z_200Vpp_alternate_no_delay_redo)
amps_all = np.array(amps_all)

In [276]:
amps_all = amps_with_fb_spin_xy_150Vpp_103kHz_z_200Vpp_alternate_no_delay_redo2[:140]

In [277]:
etha = 1/scaleZ*327*1.6e-19*voltage
to_epsilon = 1/(2*327*1.6e-19*voltage_200_1*(418)/(418+627))

In [278]:
j = 0
data = amps_all[::2,j]*etha
data2 = amps_all[1::2,j]*etha
data3 = data-0.66*data2

data_1 = np.abs(amps_all[::2,1])*etha
data2_1 = np.abs(amps_all[1::2,1])*etha
data3_1 = data_1+0.66*0.66*data2_1

_, ax= plt.subplots(figsize=(9.5,4))
# ax.scatter(range(len(data)), data, label='200Vpp-z-plus, xz-spin-50Vpp-93kHz')
# ax.scatter(range(len(data2)), data2, label='200Vpp-z-minus, xz-spin-50Vpp-93kHz', alpha=0.6)
ax.scatter(range(len(data3)), data3, label='200Vpp-z, A parameter', alpha=0.6)

# ax.scatter(range(len(data4)), data4, label='20V-x, xy-spin-30V, w/o height fb')
ax.plot(np.arange(0,2000), np.arange(0,2000)*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, electrode 1: mean, std: ', np.mean(data), np.std(data)/np.sqrt(len(data)))
print('1st harmonic, electrode 2: mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))
print('1st harmonic, A parameter: mean, std: ', np.mean(data3)*to_epsilon*4, np.std(data3)*to_epsilon/np.sqrt(len(data3)))
print()
print('2nd harmonic, electrode 1: mean, std: ', np.mean(data_1), np.std(data_1)/np.sqrt(len(data_1)))
print('2nd harmonic, electrode 2: mean, std: ', np.mean(data2_1), np.std(data2_1)/np.sqrt(len(data2_1)))
print('2nd harmonic, B parameter: mean, std: ', np.mean(data3_1)*to_epsilon, np.std(data3_1)*to_epsilon/np.sqrt(len(data3_1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, electrode 1: mean, std:  -1.28118356481e-16 6.3480503803e-18
1st harmonic, electrode 2: mean, std:  -1.87638897789e-16 6.35225597993e-18
1st harmonic, A parameter: mean, std:  -0.00214102786414 0.000831088849787

2nd harmonic, electrode 1: mean, std:  2.75729924772e-16 1.55028669544e-17
2nd harmonic, electrode 2: mean, std:  6.00047851978e-16 3.19819295305e-17
2nd harmonic, B parameter: mean, std:  0.0672231768176 0.00285440488067


In [49]:
1.83/0.66

2.7727272727272725

In [716]:
print('1st harmonic, A parameter: mean, std: ', np.mean(data3)*to_epsilon, np.std(data3)*to_epsilon/np.sqrt(5000))

1st harmonic, A parameter: mean, std:  -5.54622412598e-05 0.000123021912647


In [751]:
data3_all = np.array(list(data3*to_epsilon*1.25)+list(data33*to_epsilon))
len(data3_all)

3450

In [245]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

n_bins = 21
range_pram = 0.03
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data3*to_epsilon, bins=n_bins, range=(-range_pram, range_pram), label='125Vpp+150Vpp, 93kHz')
# entries3, bins, patches = ax.hist(data3, bins=n_bins, range=(-range_pram, range_pram), label='150Vpp, 77kHz', alpha=0.6)
ax.set(xlabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [246]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

titles = ['', '', '']

for i,ee in enumerate([entries]):
    x = bins_fit
    y= np.array(ee)
    sigma_bin = np.sqrt(ee)
    
    n = len(x)                          #the number of data
    mean = 0                  #note this correction
    sigma = 0.023       #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print(titles[i], ": mean: ", popt[1], " ,std (on the mean): ", np.sqrt(pcov)[1,1], " ,sigma: ", popt[2]/np.sqrt(len(data3)))
    print('\nNeutrality: ', 1/1.26e14*popt[1])

 : mean:  -3.89407344248e-06  ,std (on the mean):  0.000105072167829  ,sigma:  -0.000148589015239

Neutrality:  -3.09053447816e-20


/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in sqrt


In [119]:
datasets = [amps_wo_fb_spin_zx_75V_x_20V,amps_wo_fb_spin_zx_100V_x_25V, amps_wo_fb_spin_zx_150V_x_25V]
voltages = [voltage20, voltage25, voltage25]
spin_voltage = [75,100, 150]
means = []
errors = []
for dat,vv in zip(datasets, voltages):
    means.append(np.mean(dat[:,0])/scale*voltage/vv)
    errors.append(np.std(dat[:,0])/scale*voltage/vv/np.sqrt(len(dat[:,0])))

_,ax = plt.subplots()
ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='zx plane spin')
ax.set(xlabel='Spin voltage', ylabel=r'$\epsilon$', xlim=(20,160), ylim=(-0.001,0.01))

datasets = [amps_wo_fb_spin_xy_30V_x_20V,amps_wo_fb_spin_xy_75V_x_20V, amps_wo_fb_spin_xy_100V_x_20V]
voltages = [voltage20, voltage20, voltage20]
spin_voltage = [30,75, 100]
means2 = []
errors2 = []
for dat,vv in zip(datasets, voltages):
    means2.append(np.mean(dat[:,0])/scale*voltage/vv)
    errors2.append(np.std(dat[:,0])/scale*voltage/vv/np.sqrt(len(dat[:,0])))

ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='xy plane spin')
ax.plot(np.arange(20,160), np.arange(20,160)*0, '--r')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [384]:
j = 0
data = amps_with_fb_spin_xy_100V_x_20V_delay_2[:,j]/scale*voltage/voltage20_1
data2 = amps_wo_fb_spin_zx_100V_x_25V[:,j]/scale*voltage/voltage25

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data, label='20V-x, xy-spin-100V, w/ delay, w/ height fb')
ax.scatter(range(len(data2)), data2, label='25V-x, xy-spin-100V, w/ delay, w/o height fb')

ax.plot(np.arange(0,3000), np.arange(0,3000)*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean, std: ',np.mean(data), np.std(data)/np.sqrt(len(data)))
print('mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean, std:  0.00543937010689 0.000832138675641
mean, std:  0.00364790172652 0.000627235885947


In [181]:
datasets = [amps_wo_fb_spin_xy_100V_x_20V2[1200:],amps_wo_fb_spin_xy_100V_x_50V_delay2, amps_wo_fb_spin_xy_100V_x_100V_delay2]
spin_voltage = [20,50, 100]
means = []
errors = []
for dat,vv in zip(datasets, voltages):
    means.append(np.mean(dat))
    errors.append(np.std(dat)/np.sqrt(len(dat)))

_,ax = plt.subplots()
ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='xy plane spin, 100V')
ax.plot(np.arange(15,110), np.arange(15,110)*0, '--r')
ax.set(xlabel='Oscillating voltage', ylabel=r'$\epsilon$', xlim=(15,110), ylim=(-0.001,0.004))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
def get_dcV(i):
    
    fname = r'/data/old_trap/20210715/bead2/neutrality/no_height_fb_electrodes_x_20V_41_Hz_spin_xy_30V_50kHz_files_3000/turbombar_powfb_xyzcool_'+str(i)+'.h5'
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    return np.mean(bb.other_data[:,5000:85000], axis=1)*100

In [146]:
get_dcV(0)

array([-3.79819842, -0.56254724, -0.57267787,  0.66225783,  0.92968812,
       -0.39438807, -0.0508512 ,  0.22620381])

In [144]:
voltages_elec3_30V_spin = np.array(Parallel(n_jobs=40)(delayed(get_dcV)(i) for i in tqdm(range(3000))))


100%|██████████| 3000/3000 [00:09<00:00, 314.37it/s]

In [313]:
i=1
fname = r'/data/old_trap/20210817/bead1/neutrality/height_fb_0_7_x_41Hz_50V_delay_10s__spin_xz_100V_50kHz_files_3000/x_neutrality_50V_41Hz_'+str(i)+'.h5'
    
drive_freq = 41
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
    
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[3])[::decimate]
m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]

_,ax = plt.subplots()
ax.scatter(range(50000), bb.other_data[3])

NameError: name 'fit_kwargs' is not defined